In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import cmocean.cm as cmo
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import scipy
from datetime import datetime,timedelta
# from xmovie import Movie
import nfft
# import xrft
import cmath
import matplotlib as mpl
from matplotlib.lines import Line2D
import math
from math import radians, cos, sin, asin, sqrt
# import eofs.xarray as xeof

%matplotlib inline

In [2]:
def demean_xarray(da,dim):
    return da - da.mean(dim=dim),da.mean(dim=dim)

def detrend_xarray(da):
    '''
    apply detrend along time axis of 2D Xarray
    see polynomial_detrend for details about detrending
    '''
    dt = xr.apply_ufunc(
                polynomial_detrend,
                da,
                input_core_dims=[['time']],
                output_core_dims=[['time','dim0']],
                vectorize=True,
                output_dtypes=[da.dtype],
                dask="parallelized",
            )
    if 'x' in da.dims:
        dt = dt.transpose('time','y','x','dim0')
    elif 'lon' in da.dims:
        dt = dt.transpose('time','lat','lon','dim0')
    
    return dt.isel(dim0=0),dt.isel(dim0=1)

def polynomial_detrend(da,order=1):
    '''
    detrend of non uniform data on a uniform grid with nan values
    
    change order to increase order of fittet polynomial, default is 1, so linear
    
    returns detrended data and trend values
    '''
    ds = da.copy()
    mask = ~np.isnan(ds)
    if mask.sum() == 0:
        return np.stack((ds,ds),axis=-1)
    else:
        ds_masked = ds[mask]
        time = np.arange(0,len(ds))
        time_masked = time[mask]
        coeff = np.polyfit(time_masked, ds_masked, order)
        trend_nonan = np.polyval(coeff, time_masked)
        detrended = ds_masked - trend_nonan
        ds[mask] = detrended
        trend = np.copy(ds)
        trend[mask] = trend_nonan
        
        return np.stack((ds.data,trend),axis=-1)

In [3]:
def spectrum1(h, dt=1):
    """
    First cut at spectral estimation: very crude.
    
    Returns frequencies, power spectrum, and
    power spectral density.
    Only positive frequencies between (and not including)
    zero and the Nyquist are output.
    """
    nt = len(h)
    npositive = nt//2
    pslice = slice(1, npositive)
    freqs = np.fft.fftfreq(nt, d=dt)[pslice] 
    ft = np.fft.fft(h)[pslice]
    psraw = np.abs(ft) ** 2
    # Double to account for the energy in the negative frequencies.
    psraw *= 2
    # Normalization for Power Spectrum
    psraw /= nt**2
    # Convert PS to Power Spectral Density
    psdraw = psraw * dt * nt  # nt * dt is record length
    return freqs, psraw, psdraw

In [4]:
def nufft(data,xarray_apply=True):
    '''
    does a non-uniform fast fourier transform on data with a uniform grid but nan values in it
    
    returns freq: frequency in cycles per timestep (cph for hourly data)
            f_k: amplitude for each wavenumber k as a complex number
            ps: power spectrum
            psd: power spectral density
            
    xarray_apply is used for the spectral_analysis function for apply_ufunc along time dimension for each point of spatial array
        if you just want to do it for one array time series, set it to False
    
    function is taken from https://github.com/jakevdp/nfft
    '''
    mask = ~np.isnan(data)
    
    N_freq = len(data)
    k = -N_freq//2 + np.arange(N_freq)
    
    data_masked = (data)[mask]
    t = np.linspace(0, 1,N_freq)[mask]
    
    f_k = nfft.nfft_adjoint(t,data_masked,N_freq)
    ps = np.abs(f_k)**2/N_freq**2
    psd = ps * N_freq
    freq = k/N_freq
    if xarray_apply == True:
        t_return = np.linspace(0, 1,N_freq)
        t_return[~mask] = np.nan
        return np.stack((freq,f_k,ps,psd,t_return),axis=-1)
    elif xarray_apply==False:
        return freq,f_k,ps,psd,t

In [5]:
def spectral_analysis(da):
    '''
    apply the nufft on each spatial point of a 2D dataset along the timeseries
    returns xarray DataSet with frequency spectrum, powerspectrum, power spectral density for each point
    t is masked time array without nans, necessary for reconstruction of time series from frequency spectrum
    '''
    dt = xr.apply_ufunc(
                nufft,
                da,
                input_core_dims=[['time']],
                output_core_dims=[['freq','dim0']],
                vectorize=True,
                output_dtypes=['complex'],
                dask="parallelized",
                dask_gufunc_kwargs={'output_sizes':{'freq':744,'dim0':5}}
            )
    if 'x' in da.dims:
        dt = xr.Dataset(coords={
            'lat':(['y','x'],dt['lat'].data),
            'lon':(['y','x'],dt['lon'].data),
            'freq':np.real(dt[0,0,:,0].data)
        },data_vars={
            'f_k':(['y','x','freq'],dt.isel(dim0=1).data),
            'ps':(['y','x','freq'],np.real(dt.isel(dim0=2).data)),
            'psd':(['y','x','freq'],np.real(dt.isel(dim0=3).data)),
            't':(['y','x','freq'],np.real(dt.isel(dim0=4).data))


        })
        dt = dt.transpose('freq','y','x')
    elif 'lat' in da.dims:
        dt = xr.Dataset(coords={
            'lat':(dt['lat'].data),
            'lon':(dt['lon'].data),
            'freq':np.real(dt[0,0,:,0].data)
        },data_vars={
            'f_k':(['lat','lon','freq'],dt.isel(dim0=1).data),
            'ps':(['lat','lon','freq'],np.real(dt.isel(dim0=2).data)),
            'psd':(['lat','lon','freq'],np.real(dt.isel(dim0=3).data)),
            't':(['lat','lon','freq'],np.real(dt.isel(dim0=4).data))


        })
        dt = dt.transpose('freq','lat','lon')
    # dt = dt.where(dt!=0)
    return dt

# hann = xr.DataArray(coords={'time':velocity.time.values},data=np.hanning(744))
# spectral_u_hann = spectral_analysis(velocity.u_demeaned*hann)

In [6]:
def peak_finder(freq_data,ps_data,freq_min,freq_max,peak_height):
    '''
    find amplitude and frequency of maximum peak in frequency range of spectrum
    freq_data, ps_data: frequency values and powerspectrum
    freq_min, freq_max: bounds of frequency range
    peak_height: threshold for peak to detect
    
    if you want to detect on frequency amplitudes or power spectral density, adjust peak_height accordingly, should work
    
    returns maximum peak and according frequency
    '''
    
    freq_mask = np.array([freq_data > freq_min]) & np.array([freq_data < freq_max])
    freq_mask = np.squeeze(freq_mask)
    peaks,peak_heights = scipy.signal.find_peaks(ps_data[freq_mask],height=peak_height)
    freq_peaks = freq_data[freq_mask][peaks]
    if len(peak_heights['peak_heights']) != 0:
        peak_max = max(peak_heights['peak_heights'])
        freq_max = freq_peaks[peak_heights['peak_heights'] == peak_max]
    else:
        peak_max,freq_max = np.nan,np.nan
    
    return np.array([freq_max,peak_max])

In [7]:
def find_peaks_xr(data,freq_min,freq_max,peak_height):
    '''
    apply peak_finder for each point of spatial xarray
    '''
    ds = xr.apply_ufunc(
                peak_finder,
                data.freq,
                data.ps,
                freq_min,
                freq_max,
                peak_height,
                input_core_dims=[['freq'],['freq'],[],[],[]],
                output_core_dims=[['peak']],
                vectorize=True,
                output_dtypes=['float'],
                dask="parallelized",
                dask_gufunc_kwargs={'output_sizes':{'peak':2}}
            )
    return ds

In [8]:
def peak_finder_rot(freq_data,ps_data,freq_min,freq_max,peak_height):
    '''
    find amplitude and frequency of maximum peak in frequency range of rotary spectrum
    freq_data, ps_data: frequency values and powerspectrum
    freq_min, freq_max: bounds of frequency range
    peak_height: threshold for peak to detect
    
    if you want to detect on frequency amplitudes or power spectral density, adjust peak_height accordingly, should work
    
    returns maximum peak and according frequency within combined positive and negative range
    '''
    freq_mask = (np.array([freq_data > freq_min]) & np.array([freq_data < freq_max])) + (np.array([freq_data < -freq_min]) & np.array([freq_data > -freq_max]))
    freq_mask = np.squeeze(freq_mask)
    peaks,peak_heights = scipy.signal.find_peaks(ps_data[freq_mask],height=peak_height)
    freq_peaks = freq_data[freq_mask][peaks]
    if len(peak_heights['peak_heights']) != 0:
        peak_max = max(peak_heights['peak_heights'])
        freq_max = abs(freq_peaks[peak_heights['peak_heights'] == peak_max])
    else:
        peak_max,freq_max = np.nan,np.nan
    
    return np.array([freq_max,peak_max])

In [9]:
def find_peaks_rot_xr(data,freq_min,freq_max,peak_height):
    '''
    apply peak_finder_rot for each point of spatial xarray
    '''
    ds = xr.apply_ufunc(
                peak_finder_rot,
                data.freq,
                data.ps,
                freq_min,
                freq_max,
                peak_height,
                input_core_dims=[['freq'],['freq'],[],[],[]],
                output_core_dims=[['peak']],
                vectorize=True,
                output_dtypes=['float'],
                dask="parallelized",
                dask_gufunc_kwargs={'output_sizes':{'peak':2}}
            )
    return ds

In [10]:
def find_nearest(array, value):
    '''
    finds value and idx of value in array that is closest to the looked for value
    '''
    array = np.asarray(array)
    idx = np.unravel_index(np.abs(array - value).argmin(), array.shape)
    return array[idx],idx

In [11]:
def ellipse_details(fm,fp):
    '''
    calculates major and minor axis as well as azimuth angle of spectral ellipse from positive and negative frequency amplitudes
    see Walden2012 for mathematical insights
    '''
    Rp = np.abs(fp) + np.abs(fm)
    Rm = np.abs(np.abs(fp) - np.abs(fm))

    major = np.abs(Rp + Rm)
    minor = np.abs(Rp - Rm)

    azimuth = cmath.phase(fp*fm)/2
    return major,minor,azimuth

In [12]:
def hann_window(n,T):
    '''
    calculates the frequency spectrum of a hanning window for n samples leading to n frequencies and T window size
    n has to be integer and n % 2 has to be 0
    '''
    assert n%2==0
    w = np.hanning(T)
    w = w / w.sum()
    freq_hann = np.fft.fftfreq(n)

    w_transform = np.abs(np.fft.fft(w, n=n))
    mid_index = int(n/2)
    amp_hann = np.concatenate((w_transform[mid_index:],w_transform[:mid_index]))
    freq_hann = np.concatenate((freq_hann[mid_index:],freq_hann[:mid_index]))
    return freq_hann,amp_hann

def butterworth_window(n,fc,order):
    '''
    calculates the frequency spectrum of a butterworth window for n samples leading to n frequencies and fc cut off frequency with order of butterworth filter 
    n has to be integer and n % 2 has to be 0
    fs is sample frequency in Hz, we have hourly data, so 1/3600
    '''
    
    assert n%2==0
    fs = 1/3600
    nyq = 0.5 * fs
    fc = fc * (1/3600)
    freqs = (-n//2 + np.arange(n))/n/3600

    b,a = scipy.signal.butter(N=order,Wn=fc,btype='lowpass',fs = fs)
    freq_but, amp_but = scipy.signal.freqz(b,a,fs=fs,worN=freqs)
    return freq_but*3600, amp_but

In [13]:
def drop_na(data):
    '''
    drops all nan values from 1D array
    '''
    mask = ~np.isnan(data)
    return data[mask]

In [14]:
def inverse_nufft(t,f_k):
    '''
    inverse fourier transform to get back time series from fourier components
    non-uniform discrete fourier transform
    
    returns array of reconstructed velocities on the complete time grid, so with nan values
    '''
    reconstructed = nfft.nfft(drop_na(t), f_k)/len(f_k)
    reconstructed_ontime = np.copy(t)
    reconstructed_ontime[~np.isnan(t)] = reconstructed
    return reconstructed_ontime

def inverse_nufft_window(t,f_k,window_amplitudes):
    '''
    inverse fourier transform to get back time series from fourier components
    non-uniform discrete fourier transform but with amplitude taken out of high frequencies using a hanning window, which has to be predefined
    
    returns array of reconstructed velocities on the complete time grid, so with nan values
    '''
    reconstructed = nfft.nfft(drop_na(t), f_k*window_amplitudes)/len(f_k)
    reconstructed_ontime = np.copy(t)
    reconstructed_ontime[~np.isnan(t)] = reconstructed
    return reconstructed_ontime

In [15]:
def filter_window(spectral_data,window_type,**parameters):
    '''
    apply inverse nufft on 2D xarray dataset with a frequency filter applied
    returns xarray DataArray of reconstructed filtered velocities on each point in space
    '''
    if window_type == 'hanning':
        freq,amp = hann_window(len(spectral_data.freq),parameters['T'])
    elif window_type == 'butterworth':
        freq,amp = butterworth_window(len(spectral_data.freq),parameters['fc'],parameters['order'])
        
    dt = xr.apply_ufunc(
                inverse_nufft_window,
                spectral_data.t,
                spectral_data.f_k,
                amp,
                input_core_dims=[['freq'],['freq'],['time']],
                output_core_dims=[['time']],
                vectorize=True,
                output_dtypes=[np.dtype(float)],
                dask="parallelized",
            )
    dt = dt.transpose('time','y','x').assign_coords({'time':velocity.time.values})
    return dt

In [16]:
def find_nearest(array, value):
    '''
    finds value and idx of value in array that is closest to the looked for value
    '''
    array = np.asarray(array)
    idx = np.unravel_index(np.abs(array - value).argmin(), array.shape)
    return array[idx],idx

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

def unique(sequence):
    '''
    finds unique values of array while keeping the order of values in array
    (set() orders values in increasing way)
    '''
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

def section_indices(data,point1,point2):
    '''
    get x,y indices of section between latlon point1 and point2
    
    uses haversine distance to look for closest points on latlon grid to given latlon point
    
    first defines straight latlon line between point1 and point2
    
    then looks for closest points on grids to this line
    
    straight line has higher resolution than grid, so we discard all the x,y points that appear twice
    '''
    lat = np.asarray(data.lat.data)
    lon = np.asarray(data.lon.data)
    
    if len(lat.shape) == 1 & len(lon.shape) == 1:
        lon,lat = np.meshgrid(lon,lat)
    
    lat1,lon1 = point1
    lat2,lon2 = point2
    

    f = np.frompyfunc(haversine,4,1)
    value, indices1 = find_nearest(f(lon,lat,lon1,lat1).astype(float),0)
    value, indices2 = find_nearest(f(lon,lat,lon2,lat2).astype(float),0)

    x_dist = np.abs(indices1[1] - indices2[1]) 
    y_dist = np.abs(indices1[0] - indices2[0]) 

    dist_max = max(x_dist,y_dist)

    if (lat1-lat2) == 0:
        longitudes = np.linspace(lon1, lon2, dist_max*2)
        latitudes = np.ones(dist_max*2)*lat1
    elif (lon1-lon2) == 0:
        latitudes = np.linspace(lat1, lat2, dist_max*2)
        longitudes = np.ones(dist_max*2)*lon1
    else:
        latitudes = np.linspace(lat1, lat2, dist_max*2)
        longitudes = (lon2 - lon1)/(lat2 - lat1)*(latitudes - lat1) + lon1

    # x_idx = []
    # y_idx = []
    indices = []
    
    for i in range(dist_max*2):
        dummy = find_nearest(f(lon,lat,longitudes[i],latitudes[i]).astype(float),0)[1]
        # x_idx.append(dummy[1])
        # y_idx.append(dummy[0])
        indices.append((dummy[1],dummy[0]))
    indices = unique(indices)
    x_idx = [x[0] for x in indices]
    y_idx = [x[1] for x in indices]
    
    return x_idx,y_idx,latitudes,longitudes

In [17]:
def section_distance(data):
    '''
    calculates haversine distance along section from most east point to west in increasing way
    here it is the distance from the Taiwan coast, if you set the starting point correctly
    '''
    distance = [0]
    for i in -np.arange(2,len(data.lon.data)+1):
        distance.append(haversine(data.lon.data[i],data.lat.data[i],data.lon.data[i+1],data.lat.data[i+1]))
    section_distance = np.cumsum(distance)
    return np.flip(section_distance)

In [18]:
def extract_section(data,point1,point2):
    '''
    extracts section on latlon grid between point1 and point2
    adds distance along section as coordinate
    '''
    x_idx,y_idx,latitudes,longitudes = section_indices(data,point1,point2)
    if 'x' in data.dims:
        section = data.isel(x=xr.DataArray(x_idx, dims="s"), y=xr.DataArray(y_idx, dims="s"))
    elif 'lat' in data.dims:
        section = data.isel(lon=xr.DataArray(x_idx, dims="s"), lat=xr.DataArray(y_idx, dims="s"))
        
    section = section.assign_coords({'distance':('s',section_distance(section))})
    return section

In [19]:
def calculate_initial_compass_bearing(pointA, pointB):
    """
    Calculates the bearing between two points.
    The formulae used is the following:
        θ = atan2(sin(Δlong).cos(lat2),
                  cos(lat1).sin(lat2) − sin(lat1).cos(lat2).cos(Δlong))
    :Parameters:
      - `pointA: The tuple representing the latitude/longitude for the
        first point. Latitude and longitude must be in decimal degrees
      - `pointB: The tuple representing the latitude/longitude for the
        second point. Latitude and longitude must be in decimal degrees
    :Returns:
      The bearing in degrees
    :Returns Type:
      float
    """
    if (type(pointA) != tuple) or (type(pointB) != tuple):
        raise TypeError("Only tuples are supported as arguments")

    lat1 = math.radians(pointA[0])
    lat2 = math.radians(pointB[0])

    diffLong = math.radians(pointB[1] - pointA[1])

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
            * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    # Now we have the initial bearing but math.atan2 return values
    # from -180° to + 180° which is not what we want for a compass bearing
    # The solution is to normalize the initial bearing as shown below
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing

In [20]:
def rotate_velocities(u,v,angle):
    '''
    rotates velocity vector in mathematical positive direction
    '''
    
    u_ = u * np.cos(angle) - v * np.sin(angle)
    v_ = u * np.sin(angle) + v * np.cos(angle)
    
    return u_,v_

In [21]:
def rotate_section(section,variables):
    '''
    rotate velocities on section to cross and along section velocities
    
    !!! This heavily depends on how you defined the section !!!
    
    The initial rotation angle is calculated as the compass bearing between two latlon points. I then substract 90deg and change the sign.
    This leads to a clockwise rotation with the angle being between the cartesian axis and the section. However, the
    initinal angle depends on which one is entered first, leading to a difference of 180deg.
    So check the compass bearing between your point first and think about if this leads to the velocites ending up in the way you want.
    I wanted u to be mostly eastward and v to be mostly northward
    
    '''
    point1 = (section.isel(s=0).lat.data,section.isel(s=0).lon.data)
    point2 = (section.isel(s=-1).lat.data,section.isel(s=-1).lon.data)
    angle = np.deg2rad(calculate_initial_compass_bearing(point1,point2))
    angle = angle = -(angle- np.pi/2)
    section['along_vel'],section['cross_vel'] = rotate_velocities(section[variables[0]],section[variables[1]],angle)
    return section

In [22]:
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

In [23]:
def angle_between_list(datau1,datav1,datau2,datav2):
    v_1 = [(u,v) for u,v in zip(datau1,datav1)]
    v_2 = [(u,v) for u,v in zip(datau2,datav2)]

    return np.array(list(map(angle_between,v_1,v_2)))

In [24]:
def angle_between_vectors_xarray(data,variables):
    u1,v1,u2,v2 = variables
    dt = xr.apply_ufunc(angle_between_list,
                        data[u1],
                        data[v1],
                        data[u2],
                        data[v2],
                        input_core_dims=[['time'],['time'],['time'],['time']],
                        output_core_dims=[['time']],
                        vectorize=True
    )
    # dt = dt.transpose(['time','lat','lon'])
    return dt


In [25]:
def correlation(data1,data2):
    # data1 is the spatial field you want to correlate to
    # data2 is your single time series
    # calculates the correlation coefficient and p_value
    # returns the result as a numpy array, because the initial output of the function is of a weird PearsonRResult class, which doesnt work in apply_ufunc
    result = stats.pearsonr(data1,data2)
    return np.stack((result[0],result[1]), axis=-1)